## Importations

In [ ]:
# Run Keras on CPU
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = " "         # -1 if CPU

In [ ]:
# Importations
import sys
sys.path.append('utils')

from configtf2 import *
import CPutils_tf2
import feature_extraction_tf2
# import gradcam_utils_tf2

In [ ]:
# Check GPUs
tf.test.gpu_device_name()

In [ ]:
# Check version
tf.__version__

In [ ]:
# Gradio
import gradio as gr

## CNNet

In [ ]:
# Import model and initialize a testing model
from nets_tf2.firenet import FireNet
dims = (38, 31, 1)
model = FireNet.build_model(width=dims[0], height=dims[1], depth=dims[2], classes=2)
model.summary()

In [ ]:
Xtest, ytest = cload(os.path.join('..', 'data', 'test_set', 'test_set.lzma'), compression='lzma')

In [ ]:
Xtest.shape

## Load pretrained model

In [ ]:
PRETRAINED = os.path.join('..', 'pretrained_models')
model.load_weights(os.path.join(PRETRAINED, 'M2.h5'))

In [ ]:
# Sanity check
model.predict(np.expand_dims(Xtest[0:1], -1))

In [ ]:
# # Model from scratch and then load weights GRADCAM
# model2 = FireNet.build_model(width=dims[0], height=dims[1], depth=dims[2], classes=2)

# # Load weights
# model2.load_weights(os.path.join(PRETRAINED, 'MGCAM.h5'))
# for i, layer in enumerate(model2.layers):
#     layer._name  = 'layer_' + str(i)
# model2.summary()

In [ ]:
# # Load pre-trained model function wrap
# def model_constructor(PRE_TRAINED, weights='MGCAM.h5'):
#     # Model from scratch and then load weights
#     from nets_tf2.firenet import FireNet
#     dims = (38, 31, 1)
#     model2 = FireNet.build_model(width=dims[0], height=dims[1], depth=dims[2], classes=2)

#     # Load weights
#     model2.load_weights(os.path.join(PRE_TRAINED, weights))
#     for i, layer in enumerate(model2.layers):
#         layer._name  = 'layer_' + str(i)

#     return model2

# # Model
# H, W = 31, 38
# model2 = gradcam_utils_tf2.build_model(model2, model_constructor=model_constructor(PRETRAINED))
# guided_model = gradcam_utils_tf2.build_guided_model(model2)

In [ ]:
# # Test
# layer = 'layer_5'
# gcam = gradcam_utils_tf2.generate_gradCAM(batch_size=1, 
#                                           layer=layer,
#                                           model=model2,
#                                           processedimages=np.expand_dims(Xtest[0:1], -1), 
#                                           rawimages=np.expand_dims(Xtest[0:1], -1) * 255.,
#                                           save=False,
#                                           showID=-1,
#                                           title='Test', )

# print("GradCAM")
# fig2 = plt.figure()
# ax2 = fig2.add_subplot(111)
# im = ax2.imshow(Xtest[0:1].squeeze() * 255., cmap='gray')
# im2 = ax2.imshow(gcam[-1], alpha=0.4, cmap='jet')
# plt.axis('off')
# plt.tight_layout()
# plt.axis("tight")  
# plt.axis("image")  
# plt.show()
# plt.close()

## Evaluation online

In [ ]:
# Gradio function 
def image_classifier(landscape):

    # Gray scale
    landscape = landscape[:,:,0] 
    print("Landscape shape:", landscape.shape)
    
    # Debug figure
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.imshow(landscape, cmap='gray')
    plt.axis('off')
    plt.tight_layout()
    plt.axis("tight")  
    plt.axis("image")  
    plt.show()
    plt.close()

    # Normalize
    landscape = landscape / 255.
    landscape = landscape.reshape(1, 31, 38)
    
    # Predictions
    y_hat = model.predict(np.expand_dims(landscape, -1))
    
    
    # GradCAM
#     layer = 'layer_5'
#     gcam = gradcam_utils_tf2.generate_gradCAM(batch_size=1, 
#                                               layer=layer,
#                                               model=model2,
#                                               processedimages=np.expand_dims(landscape, -1), 
#                                               rawimages=np.expand_dims(landscape, -1) * 255.,
#                                               save=False,
#                                               showID=-1,
#                                               title='')
#     fig2 = plt.figure()
#     ax2 = fig2.add_subplot(111)
#     im = ax2.imshow(landscape.squeeze() * 255., cmap='gray')
#     im2 = ax2.imshow(gcam[-1], alpha=0.4, cmap='jet')
#     plt.axis('off')
#     plt.tight_layout()
#     plt.axis("tight")  
#     plt.axis("image")  
#     plt.show()
#     plt.close()
    
    
    # Return label and attention map
    return {'no fire': float(y_hat[0][0]), 'fire': float(y_hat[0][1])} #, fig

# Create interface
iface = gr.Interface(
    image_classifier, 
    gr.inputs.Image(shape=(38, 31)), 
    [gr.outputs.Label(num_top_classes=2)], # gr.outputs.Image(plot=True, label="Attention Map")],
    capture_session=True,
    interpretation="default",
    title='Deep Fire Topology (DFT)',
    description='Online version of a pretrained model following DFT structure',
    server_port=7861,
)


In [ ]:
# Launch it
iface.launch(share=False)